In [1]:
import numpy as np
import pandas as pd

### **[ 전처리 1 : 코일 재공 정보에 관한 데이터프레임 생성 및 그룹핑 ]**
#### **ANN 재공현황.csv 데이터 불러오기**

In [2]:
coil_mip_df = pd.read_csv("./data/ANN 재공현황.csv")
coil_mip_df[:3]

,BUS_CD,COIL_NO,LOT_NO,BASIC_COIL_NO,HOTCOI_NO,MOTCOI_NO,TRK_NO,EMG_CD,AFTEMG_CD,PKG_NO,...,CRT_DT,CRTCHR_NO,UPD_DT,UPDCHR_NO,COILEND_DT,SEMIPKG_YN,SEMIPKG_CD,PKG_CD,LEV2SE_YN,ZON_CD
0,1173DK,07KC1022111A,2007KC1022,07KC1022111,92311302,07KC1022111,NaN,1282N,NaN,NaN,...,2007-03-06 0:00,MGTEST,2019-07-12 14:50,이승용,NaN,NaN,NaN,NaN,E,1510P
1,1173DK,07KC1022211A,2007KC1022,07KC1022211,92311302,07KC1022211,NaN,1282N,NaN,NaN,...,2007-03-07 0:00,MGTEST,2013-01-16 14:26,김덕림,NaN,NaN,NaN,NaN,E,1510P
2,1173DK,07KC1022311A,2007KC1022,07KC1022311,92311302,07KC1022311,NaN,1282N,NaN,NaN,...,2007-03-07 0:00,MGTEST,2020-03-18 22:52,김백수,NaN,NaN,NaN,NaN,E,1510P


#### **ANN 재공현황.csv에서 필요한 열만 선정**


In [3]:
coil_mip_df_copy = coil_mip_df.copy()
coil_mip_df_selected_columns = coil_mip_df_copy[["COIL_NO","PRDSPC_ID", "PNSPRC_CD", "WDT_LEN","WGT_WGT","IND_CD","OUD_LEN","STL_CD","EMG_CD"]]
coil_mip_df_selected_columns.head()

,COIL_NO,PRDSPC_ID,PNSPRC_CD,WDT_LEN,WGT_WGT,IND_CD,OUD_LEN,STL_CD,EMG_CD
0,07KC1022111A,SCP1-H-B,AN10,392.0,5688,1260508,1616,10021,1282N
1,07KC1022211A,SCP1-H-B,AN10,392.0,5688,1260508,1616,10021,1282N
2,07KC1022311A,SCP1-H-B,AN10,392.0,5688,1260508,1616,10021,1282N
3,07KK1D22111,SCP1-H-B,AN10,1228.0,17479,1260400,1571,10021,NaN
4,09KA1B77311,SCP1-H-B,AN11,309.0,4811,1260508,1668,10021,1282N


*EMG_CD : 해당 재고의 긴급 작업 해당 유무
- 1282Y, 1282S : 긴급O
- 1282N : 긴급X

결측치 확인

In [4]:
coil_mip_df_selected_columns.isnull().sum()

COIL_NO      0
PRDSPC_ID    0
PNSPRC_CD    0
WDT_LEN      0
WGT_WGT      0
IND_CD       0
OUD_LEN      0
STL_CD       0
EMG_CD       1
dtype: int64

#### **긴급 Column이 Nan인 결측치는 1282N으로 채우기**

In [5]:
coil_mip_df_selected_columns['EMG_CD'].fillna('1282N', inplace = True)

C:\Users\LG\AppData\Local\Temp\ipykernel_2176\2204557711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coil_mip_df_selected_columns['EMG_CD'].fillna('1282N', inplace = True)


결측치 처리 확인

In [6]:
coil_mip_df_selected_columns.isnull().sum()

COIL_NO      0
PRDSPC_ID    0
PNSPRC_CD    0
WDT_LEN      0
WGT_WGT      0
IND_CD       0
OUD_LEN      0
STL_CD       0
EMG_CD       0
dtype: int64

## 코일 무게 단위 바꾸기
### kg -> Ton

In [9]:
coil_mip_df_selected_columns["WGT_WGT"] = coil_mip_df_selected_columns["WGT_WGT"]/1000

C:\Users\LG\AppData\Local\Temp\ipykernel_2176\2511156382.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coil_mip_df_selected_columns["WGT_WGT"] = coil_mip_df_selected_columns["WGT_WGT"]/1000


긴급한 데이터 비율 확인

In [10]:
coil_mip_df_selected_columns["EMG_CD"].value_counts()

1282N    329
1282Y     48
1282S     18
Name: EMG_CD, dtype: int64

#### **special_cycle.csv 파일 불러오기**
- 결측치 채우기

In [11]:
special_cycle = pd.read_csv("./data/special_cycle.csv")
special_cycle_copy = special_cycle.fillna("-")
special_cycle_copy[:3]

,고객사,제품 규격 코드,ANN0,ANN1,ANN2,ANN3
0,한국보그워너티에스,DK800,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용
1,UNIT TIDC INDIA,S700MC,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용
2,FAURECIA(CHINA) / BROSE CHINA,H800LA,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용


- ANNN 재공현황.csv와 합치기 위해 groupby를 통해 데이터 행 조정 1

In [12]:
special_cycle_copy.rename(columns={ special_cycle_copy.columns[1] : 'PRDSPC_ID',
                                    special_cycle_copy.columns[2] : 'AN10',
                                    special_cycle_copy.columns[3] : 'AN11',
                                    special_cycle_copy.columns[4] : 'AN12',
                                    special_cycle_copy.columns[5] : 'AN13',}, inplace=True)
special_cycle_copy.set_index(['고객사','PRDSPC_ID'],inplace=True)
special_cycle_copy.head()

,,AN10,AN11,AN12,AN13
고객사,PRDSPC_ID,,,,
한국보그워너티에스,DK800,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용
UNIT TIDC INDIA,S700MC,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용
FAURECIA(CHINA) / BROSE CHINA,H800LA,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용
DYNAX CORP.,S55C-PA,-,620,620,620
한국보그워너티에스,S45CM-R,-,620,620,620


- ANNN 재공현황.csv와 합치기 위해 groupby를 통해 데이터 행 조정 2

In [13]:
special_cycle_for_merge = pd.DataFrame(special_cycle_copy.stack()).reset_index()
special_cycle_for_merge.rename(columns={'level_2' : 'PNSPRC_CD', 0 : 'cycle'}, inplace=True)
special_cycle_for_merge

,고객사,PRDSPC_ID,PNSPRC_CD,cycle
0,한국보그워너티에스,DK800,AN10,작업지시서 소둔 Cycle 적용
1,한국보그워너티에스,DK800,AN11,작업지시서 소둔 Cycle 적용
2,한국보그워너티에스,DK800,AN12,작업지시서 소둔 Cycle 적용
3,한국보그워너티에스,DK800,AN13,작업지시서 소둔 Cycle 적용
4,UNIT TIDC INDIA,S700MC,AN10,작업지시서 소둔 Cycle 적용
...,...,...,...,...
111,(주)아스픽 / (주)엠알인프라오토,AISI4140M-S,AN13,750
112,유신정밀공업(주) / (주)화진금속,50CRV4M-S,AN10,750
113,유신정밀공업(주) / (주)화진금속,50CRV4M-S,AN11,750
114,유신정밀공업(주) / (주)화진금속,50CRV4M-S,AN12,750


#### **전처리된 두 df을 하나의 최종 df로 merge**

#### **Coil grouping 기준 : ANN차수, 긴급 유무**
- **ANN 재공현황.csv**와 **cycle에 관한 정보 파일**을 합쳐서 솔버에 적용시켜줄 데이터프레임을 생성

In [14]:
merged_df= pd.merge(coil_mip_df_selected_columns, special_cycle_for_merge, on=['PRDSPC_ID', 'PNSPRC_CD'], how='left')
#merged_data = pd.merge(sorted_by_ANN, special_cycle_test_copy, left_on='PRDSPC_ID', right_on='제품 규격 코드',how='left')
merged_df = merged_df[["COIL_NO","PRDSPC_ID", "PNSPRC_CD", "cycle", "WDT_LEN","WGT_WGT","IND_CD","OUD_LEN","STL_CD","EMG_CD","고객사"]]
merged_df

,COIL_NO,PRDSPC_ID,PNSPRC_CD,cycle,WDT_LEN,WGT_WGT,IND_CD,OUD_LEN,STL_CD,EMG_CD,고객사
0,07KC1022111A,SCP1-H-B,AN10,NaN,392.0,5.688,1260508,1616,10021,1282N,NaN
1,07KC1022211A,SCP1-H-B,AN10,NaN,392.0,5.688,1260508,1616,10021,1282N,NaN
2,07KC1022311A,SCP1-H-B,AN10,NaN,392.0,5.688,1260508,1616,10021,1282N,NaN
3,07KK1D22111,SCP1-H-B,AN10,NaN,1228.0,17.479,1260400,1571,10021,1282N,NaN
4,09KA1B77311,SCP1-H-B,AN11,NaN,309.0,4.811,1260508,1668,10021,1282N,NaN
...,...,...,...,...,...,...,...,...,...,...,...
390,22J120212111A,SAE1055M-R,AN10,NaN,368.0,5.556,1260508,1645,10022,1282N,NaN
391,22J120213111A,SAE1055M-R,AN10,NaN,317.0,4.786,1260508,1645,10022,1282N,NaN
392,22J12072111,SAE1070M-S,AN10,NaN,1240.0,21.250,1260762,1833,10022,1282N,NaN
393,22J13014111,100CR6,AN10,750*Max,1090.0,21.470,1260762,1943,10023,1282N,(주)지에스앤티 / 한국엔에스케이


### **merged_df의 cycle열 결측치 보완**

In [15]:
merged_df.isnull().sum()

COIL_NO        0
PRDSPC_ID      0
PNSPRC_CD      0
cycle        307
WDT_LEN        0
WGT_WGT        0
IND_CD         0
OUD_LEN        0
STL_CD         0
EMG_CD         0
고객사          307
dtype: int64

- cycle열의 결측치는 일반cycle에 관한 내용. 기존에 존재하는 데이터로 보완

**[ 10021 : 일반강 ]**
- ANN 0차 : X
- ANN 1차 이상 : 620

**[ 10022 : SC강 ]**
- ANN 0차 : 725
- ANN 1차 이상 : 700

**[ 10023 : SK강 ]**
- ANN 0차 : 735
- ANN 1차 이상 : 735


In [16]:
merged_df['cycle'] = np.where((merged_df['PNSPRC_CD'] == 'AN10') & (merged_df['STL_CD'] == 10021) & (merged_df['cycle'].isnull()), "-", merged_df['cycle'])
merged_df['cycle'] = np.where((merged_df['PNSPRC_CD'] != 'AN10') & (merged_df['STL_CD'] == 10021) & (merged_df['cycle'].isnull()), "620", merged_df['cycle'])

merged_df['cycle'] = np.where((merged_df['PNSPRC_CD'] == 'AN10') & (merged_df['STL_CD'] == 10022) & (merged_df['cycle'].isnull()), "725", merged_df['cycle'])
merged_df['cycle'] = np.where((merged_df['PNSPRC_CD'] != 'AN10') & (merged_df['STL_CD'] == 10022) & (merged_df['cycle'].isnull()), "700", merged_df['cycle'])

merged_df['cycle'] = np.where((merged_df['PNSPRC_CD'] == 'AN10') & (merged_df['STL_CD'] == 10023) & (merged_df['cycle'].isnull()), "735", merged_df['cycle'])
merged_df['cycle'] = np.where((merged_df['PNSPRC_CD'] != 'AN10') & (merged_df['STL_CD'] == 10023) & (merged_df['cycle'].isnull()), "735", merged_df['cycle'])

In [17]:
merged_df.isnull().sum()

COIL_NO        0
PRDSPC_ID      0
PNSPRC_CD      0
cycle          0
WDT_LEN        0
WGT_WGT        0
IND_CD         0
OUD_LEN        0
STL_CD         0
EMG_CD         0
고객사          307
dtype: int64

#### **cycle열의 '-' 값을 가진 행은 drop**

In [18]:
drop_column_index = merged_df[merged_df["cycle"].str.contains("-")].index
merged_df.drop(drop_column_index, inplace=True)
merged_df

,COIL_NO,PRDSPC_ID,PNSPRC_CD,cycle,WDT_LEN,WGT_WGT,IND_CD,OUD_LEN,STL_CD,EMG_CD,고객사
4,09KA1B77311,SCP1-H-B,AN11,620,309.0,4.811,1260508,1668,10021,1282N,NaN
5,21M12B721111A,S45CM-S,AN11,700,157.0,1.376,1260400,1290,10022,1282N,NaN
6,21M12D061111A,50CRV4M-S,AN10,750,248.0,2.091,1260508,1275,10022,1282S,유신정밀공업(주) / (주)화진금속
7,22A122851111A,S55CM-R,AN10,725,410.0,5.121,1260508,1511,10022,1282N,NaN
8,22A122852111A,S55CM-R,AN10,725,410.0,5.121,1260508,1511,10022,1282N,NaN
...,...,...,...,...,...,...,...,...,...,...,...
390,22J120212111A,SAE1055M-R,AN10,725,368.0,5.556,1260508,1645,10022,1282N,NaN
391,22J120213111A,SAE1055M-R,AN10,725,317.0,4.786,1260508,1645,10022,1282N,NaN
392,22J12072111,SAE1070M-S,AN10,725,1240.0,21.250,1260762,1833,10022,1282N,NaN
393,22J13014111,100CR6,AN10,750*Max,1090.0,21.470,1260762,1943,10023,1282N,(주)지에스앤티 / 한국엔에스케이


#### **외경 열에서 이상치 제거**

In [19]:
merged_df["IND_CD"].value_counts()

1260508    195
1260610    192
1260762      3
1260400      1
Name: IND_CD, dtype: int64

In [20]:
drop_index = merged_df[(merged_df["IND_CD"] == 1260762)|(merged_df["IND_CD"] == 1260400)].index
merged_df.drop(drop_index, inplace=True)

In [21]:
merged_df["IND_CD"].value_counts()

1260508    195
1260610    192
Name: IND_CD, dtype: int64

- 내경 데이터 형태 수정
- 1260508 -> 508 : 앞 4자리 슬라이싱

In [22]:
merged_df = merged_df.astype({"IND_CD" : "str"})
merged_df.head()

,COIL_NO,PRDSPC_ID,PNSPRC_CD,cycle,WDT_LEN,WGT_WGT,IND_CD,OUD_LEN,STL_CD,EMG_CD,고객사
4,09KA1B77311,SCP1-H-B,AN11,620,309.0,4.811,1260508,1668,10021,1282N,NaN
6,21M12D061111A,50CRV4M-S,AN10,750,248.0,2.091,1260508,1275,10022,1282S,유신정밀공업(주) / (주)화진금속
7,22A122851111A,S55CM-R,AN10,725,410.0,5.121,1260508,1511,10022,1282N,NaN
8,22A122852111A,S55CM-R,AN10,725,410.0,5.121,1260508,1511,10022,1282N,NaN
9,22A122853111A,S55CM-R,AN10,725,210.0,2.623,1260508,1511,10022,1282N,NaN


In [23]:
merged_df['IND_CD'] = merged_df['IND_CD'].str.slice(4,7)

In [24]:
merged_df["IND_CD"].value_counts()

508    195
610    192
Name: IND_CD, dtype: int64

In [25]:
merged_df.reset_index(drop=True, inplace= True)

In [26]:
merged_df

,COIL_NO,PRDSPC_ID,PNSPRC_CD,cycle,WDT_LEN,WGT_WGT,IND_CD,OUD_LEN,STL_CD,EMG_CD,고객사
0,09KA1B77311,SCP1-H-B,AN11,620,309.0,4.811,508,1668,10021,1282N,NaN
1,21M12D061111A,50CRV4M-S,AN10,750,248.0,2.091,508,1275,10022,1282S,유신정밀공업(주) / (주)화진금속
2,22A122851111A,S55CM-R,AN10,725,410.0,5.121,508,1511,10022,1282N,NaN
3,22A122852111A,S55CM-R,AN10,725,410.0,5.121,508,1511,10022,1282N,NaN
4,22A122853111A,S55CM-R,AN10,725,210.0,2.623,508,1511,10022,1282N,NaN
...,...,...,...,...,...,...,...,...,...,...,...
382,22J120202111A,SAE1055M-R,AN10,725,354.0,5.027,508,1600,10022,1282N,NaN
383,22J120203111A,SAE1055M-R,AN10,725,323.0,4.587,508,1600,10022,1282N,NaN
384,22J120211111A,SAE1055M-R,AN10,725,317.0,4.786,508,1645,10022,1282N,NaN
385,22J120212111A,SAE1055M-R,AN10,725,368.0,5.556,508,1645,10022,1282N,NaN


In [27]:
merged_df.to_csv("./data/coil_information.csv", sep=",", na_rep = "NaN")

---

In [28]:
df = pd.read_csv("./data/coil_information.csv")
df.drop("Unnamed: 0", axis=1, inplace=True)
data_df = df.copy()
data_df

,COIL_NO,PRDSPC_ID,PNSPRC_CD,cycle,WDT_LEN,WGT_WGT,IND_CD,OUD_LEN,STL_CD,EMG_CD,고객사
0,09KA1B77311,SCP1-H-B,AN11,620,309.0,4.811,508,1668,10021,1282N,NaN
1,21M12D061111A,50CRV4M-S,AN10,750,248.0,2.091,508,1275,10022,1282S,유신정밀공업(주) / (주)화진금속
2,22A122851111A,S55CM-R,AN10,725,410.0,5.121,508,1511,10022,1282N,NaN
3,22A122852111A,S55CM-R,AN10,725,410.0,5.121,508,1511,10022,1282N,NaN
4,22A122853111A,S55CM-R,AN10,725,210.0,2.623,508,1511,10022,1282N,NaN
...,...,...,...,...,...,...,...,...,...,...,...
382,22J120202111A,SAE1055M-R,AN10,725,354.0,5.027,508,1600,10022,1282N,NaN
383,22J120203111A,SAE1055M-R,AN10,725,323.0,4.587,508,1600,10022,1282N,NaN
384,22J120211111A,SAE1055M-R,AN10,725,317.0,4.786,508,1645,10022,1282N,NaN
385,22J120212111A,SAE1055M-R,AN10,725,368.0,5.556,508,1645,10022,1282N,NaN


- Coil Numeber

In [29]:
coil_number = data_df["COIL_NO"].tolist()
coil_number[:5]

['09KA1B77311',
 '21M12D061111A',
 '22A122851111A',
 '22A122852111A',
 '22A122853111A']

- ANN 차수

In [30]:
ann_number = data_df["PNSPRC_CD"].tolist()
ann_number[:5]

['AN11', 'AN10', 'AN10', 'AN10', 'AN10']

- cycle

In [31]:
cycle = data_df["cycle"].tolist()
cycle[:5]

['620', '750', '725', '725', '725']

- Coil 폭

In [32]:
coil_length = data_df["WDT_LEN"].tolist()
coil_length[:5]

[309.0, 248.0, 410.0, 410.0, 210.0]

- Coil 무게

In [33]:
coil_weight = data_df["WGT_WGT"].tolist()
coil_weight[:5]

[4.811, 2.091, 5.121, 5.121, 2.623]

- Coil 내경

In [34]:
coil_inner = data_df["IND_CD"].tolist()
coil_inner[:5]

[508, 508, 508, 508, 508]

- Coil 외경

In [35]:
coil_outer = data_df["OUD_LEN"].tolist()
coil_outer[:5]

[1668, 1275, 1511, 1511, 1511]

- Coil 긴급 유무

In [36]:
coil_emergency = data_df["EMG_CD"].tolist()
coil_emergency[:5]

# ann차수, cycle을 통한 여러개의 조합들 중, 해당하는 열이 없는 조합일 때는 그냥 스킵만 해줄까 ?

['1282N', '1282S', '1282N', '1282N', '1282N']

---

### **[ 전처리 2 : 베이스 capacity 정보에 관한 데이터프레임 생성 및 그룹핑 ]**

In [37]:
base_df = pd.read_csv("./data/base_capacity_information.csv")
base_df.head()

,Maker,Base_number,Weight(Ton),Height(mm),Outer_max(mm),Outer_min(mm),Inner(mm)
0,EBNER,1,35,2800,1700,900,508
1,EBNER,2,35,2800,1700,900,508
2,EBNER,3,35,2800,1700,900,508
3,EBNER,4,35,2800,1700,900,508
4,EBNER,5,35,2800,1700,900,508


- Maker

In [38]:
base_maker = base_df['Maker'].tolist()
base_maker[:5]

['EBNER', 'EBNER', 'EBNER', 'EBNER', 'EBNER']

- Base Number

In [39]:
base_number = base_df['Base_number'].tolist()
base_number[:5]

[1, 2, 3, 4, 5]

- Base의 코일 수용 가능 무게

In [40]:
base_weight = base_df['Weight(Ton)'].tolist()
base_weight[:5]

[35, 35, 35, 35, 35]

- Base 높이

In [41]:
base_height = base_df['Height(mm)'].tolist()
base_height[:5]

[2800, 2800, 2800, 2800, 2800]

- Base 외경 최대

In [46]:
base_outer_max = base_df['Outer_max(mm)'].tolist()
base_outer_max[:5]

[1700, 1700, 1700, 1700, 1700]

- Base 외경 최소

In [47]:
base_outer_min = base_df['Outer_min(mm)'].tolist()
base_outer_min[:5]

[900, 900, 900, 900, 900]

- Base 내경

In [48]:
base_inner = base_df['Inner(mm)'].tolist()
base_inner[:5]

[508, 508, 508, 508, 508]